In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('fakeandrealdata.csv')

In [4]:
df=df.drop(columns='Unnamed: 0')

In [8]:
df=df.drop_duplicates()

In [11]:
df = df[df['Inputs'].str.contains("عتاب ") == False]

In [12]:
df

,Inputs,Label
0,أعلنت شركة “ بي.بي. آي ” ، المتخصصة في مجال تص...,0
1,فاجأ المطرب الشاب مصطفى سيد جمهوره بأغنية “ ال...,0
2,القبض على رئيس مجلس بلدة الرهوة أقدم مواطن ( 4...,0
3,تعقد شركة أرامكو السعودية لأول مرة في الرياض م...,0
5,"عادت فرقة "" "" دايسكي "" من كوريا الجنوبية الى ا...",0
...,...,...
99995,رائع وموقعه مميز . موقع الفندق بالقرب من الخور...,1
99996,رحلة مميزة بالقرب من الحرم . تعامل الموظفين كا...,1
99997,حجز لثلاث اشخاص . تسجيل الدخول للفندق و تسجيل ...,1
99998,مخيب للأمل. . تسجيل الدخول استغرق اكثر من ساعتين,1


In [13]:
df.Label.value_counts()

1    43726
0    35684
Name: Label, dtype: int64

In [14]:
class_2,class_1 = df.Label.value_counts()
c2 = df[df['Label'] == 0]
c1 = df[df['Label'] == 1]
df_2_over = c2.sample(class_2, replace=True)

In [15]:
df = pd.concat([df_2_over,c1],axis=0)

In [16]:
df.Label.value_counts()

0    43726
1    43726
Name: Label, dtype: int64

In [17]:
df

,Inputs,Label
34977,برنامج Essentials Pro 9. 0. 2. 1. 6 Portable ي...,0
46463,عرب بت - أعلنت اللجنة المنظمة لمهرجان التسوق ا...,0
36231,وكانت وزارة الاستثمار قد رفعت أسعار تلك الخدما...,0
71,قال وزير الخارجية البريطاني ، ويليام هيج ، الي...,0
42859,حجز تشيلسي فرصة التتويج بلقب دوري أبطال أوروبا...,0
...,...,...
99995,رائع وموقعه مميز . موقع الفندق بالقرب من الخور...,1
99996,رحلة مميزة بالقرب من الحرم . تعامل الموظفين كا...,1
99997,حجز لثلاث اشخاص . تسجيل الدخول للفندق و تسجيل ...,1
99998,مخيب للأمل. . تسجيل الدخول استغرق اكثر من ساعتين,1


In [18]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv02")


Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
from sklearn.model_selection import train_test_split
text = df['Inputs']
label = df['Label']


In [20]:
X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.25, stratify=label)

In [21]:
def tokenize_data(tokenizer, texts):
  """Tokenizes a list of text strings using the given BERT tokenizer.

  Args:
    tokenizer: A BERT tokenizer.
    texts: A list of text strings.

  Returns:
    A list of tokenized sentences, where each sentence is a list of token IDs.
  """

  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True, max_length=128)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [22]:
x =tokenize_data(tokenizer, X_train)

In [23]:
x1 =tokenize_data(tokenizer, X_val)

In [24]:
import keras
import transformers
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [25]:
embedding_layer = Embedding(
    input_dim=model.config.vocab_size,
    output_dim=model.config.hidden_size,
    weights=[model.embeddings.word_embeddings.weight.detach()],
    trainable=True)

In [26]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    embedding_layer,
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],)


In [28]:
model.fit(x_train, y_train, epochs=10,steps_per_epoch=100,batch_size=32)


Epoch 1/10
100/100 [==============================] - 14s 57ms/step - loss: 0.1446 - accuracy: 0.9328
Epoch 2/10
100/100 [==============================] - 5s 50ms/step - loss: 0.0468 - accuracy: 0.9841
Epoch 3/10
100/100 [==============================] - 5s 51ms/step - loss: 0.0314 - accuracy: 0.9922
Epoch 4/10
100/100 [==============================] - 5s 51ms/step - loss: 0.0294 - accuracy: 0.9916
Epoch 5/10
100/100 [==============================] - 5s 52ms/step - loss: 0.0200 - accuracy: 0.9947
Epoch 6/10
100/100 [==============================] - 5s 52ms/step - loss: 0.0215 - accuracy: 0.9925
Epoch 7/10
100/100 [==============================] - 5s 52ms/step - loss: 0.0172 - accuracy: 0.9953
Epoch 8/10
100/100 [==============================] - 5s 51ms/step - loss: 0.0171 - accuracy: 0.9947
Epoch 9/10
100/100 [==============================] - 5s 51ms/step - loss: 0.0145 - accuracy: 0.9944
Epoch 10/10
100/100 [==============================] - 5s 50ms/step - loss: 0.0119 - accur

In [29]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 2s 11ms/step - loss: 0.0165 - accuracy: 0.9950


[0.01652875542640686, 0.9949686527252197]

In [64]:
import numpy as np
text="بالطبع، يمكنني توليد كود لنموذج BiLSTM باستخدام  وKeras. هذا هو مثال على كيفية إنشاء نموذج BiLSTM باستخدام TensorFlow وKeras:"
text_p = [text]
x2 = tokenize_data(tokenizer, text_p)

# Make a prediction
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)


output = model.predict(prediction_input)




if output > 0.5:
    predicted_class = 'human written'
else:
    predicted_class = 'machine generated'

# Print the predicted class
print('Predicted class:', predicted_class,output)


1/1 [==============================] - 0s 26ms/step
Predicted class: machine generated [[0.00026796]]


In [45]:
model.save('arabicbertfordeepfaketextdetection.h5')